## Image processing
Some text explaining image processing

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

def test_plot():
    plt.plot(np.arange(10), '.-')
    
test_plot()

In [ ]:
!pip install git+https://github.com/ArcetriAdaptiveOptics/OAO24.git
from oao24 import package_data

dark_image = np.load(package_data.tuto3_folder() / "ID_110.npy")
open_loop_image = np.load(package_data.tuto3_folder() / "ID_105.npy")

In [ ]:
seeing_limited_ima = open_loop_image-dark_image
ima = np.clip(seeing_limited_ima, 0, np.inf)
plt.imshow(ima)
plt.colorbar()

## Tutorial 3 for Deconvolution
1. Create an image with objects and PSFs provided
   1. Objects: Vesta, Ganymede (to be determined) 
   2. PSFs: Papyrus-liked PSFs 
2. Using Wiener Filtering to deconvolve an image 
   1. Equations for the noise estimation will be provided in the notebook 
   2. Same with the wiener filtering equation but it is expected to create the function for wiener filtering function by themselves
3. MAP 
   1. Gradient descent method with regularisation
   2. Equation will be introduced in the class 
   3. Function will be provided in the notebook 

Tasks: 
1. Impact of S/N on deconvolution 
2. Impact of PSFs accuracy on deconvolution 


Before starting to work on deconvolution, first of all we need to understand how is an image formed.

\begin{equation}
I = H*O+N, 
\end{equation}

where $I$ is the image, $H$ is the PSF, $O$ is the object and $N$ is the noise.


Maximum A Posteriori #TODO add more descirption and equations

In [ ]:
#%% MAXIMUM A POSTERIORI (with minimisation algorithm)
class MAPcriterion:
    def __init__(self, image, psf, weights):
        self.iteration = 0
        self.criterion = 0
        self.image = image
        self.otf = fft2(psf)
        self.weights = weights
        image_less_noisy = gaussian_filter(image, 1)
        self.norma = regularization_quadratic(image_less_noisy)[0]/image.size
        self.norma *= 100 # tune hyper-parameter (empirical)
        
    def __call__(self, obj_flat):
        obj = np.reshape(obj_flat, self.otf.shape)
        obj_tf = fft2(obj)
        likeli,likeli_g = likelihood(self.image, obj_tf, self.otf, self.weights)
        regul,regul_g = regularization_quadratic(obj)
        map_fun = likeli + regul/self.norma
        map_grad = likeli_g + regul_g/self.norma
        self.criterion = map_fun
        return map_fun, map_grad.flatten()
    
    def callback(self, obj_flat):
        self.iteration += 1
        print('[%2u] %5.8g'%(self.iteration,self.criterion))